# Trainor Photometry DLC

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from scipy.signal import butter, filtfilt

In [2]:
# Define project path and get training data
data = pd.read_csv('File Path/Raw_Photometry.csv', skiprows=[0, 1])
data = data.drop(data.columns[3:8], axis=1)
#I dropped both of the header rows and any columns that weren't time, 405 raw data, and 470 raw data

In [3]:
#Give column names
data.columns = ['time','405nm','470nm']
data = data.astype('float')
data = data.dropna()

## Method of Fitting Biexponentials

In [4]:
def biexponential(x, a, b, c, d):
    return a * np.exp(b * x) + c * np.exp(d * x)

In [5]:
def iso_biexponential(df, fit='405nm'):
    """
    Fits isosbestic channel with biexponential to correct bleaching in signal channel:
    - Use least square regression to fit the isosbestic signal.

    Parameters
    ----------
    df : DataFrame
        Data to apply linear fit on.
    
    Returns
    -------
    
    """

    # define vars for biexponential model:

    X = df['time']
    Y = df[fit]
    popt, pcov = curve_fit(biexponential,X,Y,p0=(0.2,0,0.2,0),maxfev=10000)
    isoBiexp = biexponential(X,*popt)
    df[f'{fit}_biexp'] = isoBiexp
    return df
    

In [6]:
# Calculate biexponential fit for isosbestic signal to correct for bleaching
df_biexp =iso_biexponential(data, fit='405nm')

In [7]:
def fit_biexponential(df, fit='405nm', yvar='470nm'):
    

    X = df['time']
    Y = df[yvar]
    isoFitY = df[f'{fit}_biexp']
    isoBiexp = np.vstack((X,isoFitY)).T
    model = LinearRegression().fit(isoBiexp, Y)
    Ypred = model.predict(isoBiexp)

    df[f'{yvar}_biexp'] = Ypred
    dFFBiexp = 100*((Y-Ypred)/Ypred)
    df[f'{yvar}_dFF_biexp'] = dFFBiexp

    return df

In [8]:
# Predict 470 signal from biexponential of isosbestic channel
df_fit_biexp= fit_biexponential(df_biexp, fit='405nm', yvar='470nm')

In [9]:
data

,time,405nm,470nm,405nm_biexp,470nm_biexp,470nm_dFF_biexp
1,0.116283,0.108112,0.131866,0.106325,0.137080,-3.803632
2,0.124583,0.108112,0.131794,0.106325,0.137080,-3.855678
3,0.132883,0.108102,0.131676,0.106325,0.137079,-3.941281
4,0.141183,0.108056,0.131540,0.106324,0.137079,-4.040666
5,0.156704,0.107929,0.131320,0.106324,0.137078,-4.200673
...,...,...,...,...,...,...
22192,212.855492,0.105000,0.128117,0.103960,0.127058,0.833390
22193,212.863792,0.104934,0.128030,0.103960,0.127058,0.765046
22194,212.872092,0.104893,0.127971,0.103960,0.127057,0.719199
22195,212.880392,0.104902,0.127963,0.103961,0.127057,0.712710


In [10]:
#Resample to 30 samples per second

In [11]:
def resample_data(df, freq=30):
    """
    Resamples data to a specified frequency. Converts index to Timedelta,
    and uses .resample() to resample data.

    Parameters
    ----------
    df : DataFrame
        DataFrame object containing data from load_session_data()
    freq : int
        Value of frequency to resample data, by default 10.
        
    Returns
    -------
    Resampled DataFrame
    """
 
    period = round(1/freq,9) #might need to use round(, ndigits=3) if getting error with freq
    df_list = []

    # convert index to TimeDeltaIndex for resampling
    df.index = df['time']
    df.index = pd.to_timedelta(df.index, unit='S')
    df = df.resample(f'{period}S').mean()
    # interpolate if there are NaNs
#     if pd.isnull(df['Time(s)']) is True:
#         df = df.interpolate()
    df['time'] = df.index.total_seconds()  

    
    return df

In [12]:
data = resample_data(df_fit_biexp, freq=30)

In [13]:
data

,time,405nm,470nm,405nm_biexp,470nm_biexp,470nm_dFF_biexp
time,,,,,,
0 days 00:00:00.116283,0.116283,0.108095,0.131719,0.106325,0.137079,-3.910314
0 days 00:00:00.149616333,0.149616,0.107899,0.131293,0.106324,0.137078,-4.219663
0 days 00:00:00.182949666,0.182950,0.108092,0.131386,0.106323,0.137076,-4.151219
0 days 00:00:00.216282999,0.216283,0.108085,0.131280,0.106323,0.137074,-4.226924
0 days 00:00:00.249616332,0.249616,0.107848,0.130892,0.106322,0.137072,-4.508674
...,...,...,...,...,...,...
0 days 00:03:32.749614207,212.749614,0.104810,0.128235,0.103955,0.127061,0.923891
0 days 00:03:32.782947540,212.782948,0.104832,0.128184,0.103957,0.127060,0.885169
0 days 00:03:32.816280873,212.816281,0.105016,0.128213,0.103958,0.127059,0.908425


In [14]:
#Z-Score with 5 sec to 25 sec as baseline

In [15]:
data = data.drop (['405nm','470nm','405nm_biexp','470nm_biexp'], axis=1)
data

,time,470nm_dFF_biexp
time,,
0 days 00:00:00.116283,0.116283,-3.910314
0 days 00:00:00.149616333,0.149616,-4.219663
0 days 00:00:00.182949666,0.182950,-4.151219
0 days 00:00:00.216282999,0.216283,-4.226924
0 days 00:00:00.249616332,0.249616,-4.508674
...,...,...
0 days 00:03:32.749614207,212.749614,0.923891
0 days 00:03:32.782947540,212.782948,0.885169
0 days 00:03:32.816280873,212.816281,0.908425


In [16]:
baseline = data.loc[(data['time'] < 25), :]
baseline = data.loc[(data['time'] > 5), :]
pre_mean = baseline.mean()
pre_std = np.std(baseline)
norm = (data - pre_mean)/pre_std
norm['time'] = data['time']
norm.rename(columns={'470nm_dFF_biexp': 'zscore'}, inplace=True)
norm['470nm_dFF_biexp'] = data['470nm_dFF_biexp']
norm.rename(columns={'470nm_dFF_biexp': 'dFF'}, inplace=True)
norm.dropna(subset=['time'], inplace=True)
norm

/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


,time,zscore,dFF
time,,,
0 days 00:00:00.116283,0.116283,-0.981423,-3.910314
0 days 00:00:00.149616333,0.149616,-1.057486,-4.219663
0 days 00:00:00.182949666,0.182950,-1.040657,-4.151219
0 days 00:00:00.216282999,0.216283,-1.059271,-4.226924
0 days 00:00:00.249616332,0.249616,-1.128548,-4.508674
...,...,...,...
0 days 00:03:32.749614207,212.749614,0.207214,0.923891
0 days 00:03:32.782947540,212.782948,0.197693,0.885169
0 days 00:03:32.816280873,212.816281,0.203411,0.908425


In [17]:
norm.to_csv('File Path/Analyzed_Photometry.csv')